# Damage-plasticity model with generic time stepping
This notebook is a work in progress on an abstract and general implementation of time integration algorithm for general damage-plasticity modes. It serves for the development of a package that can be configured by specifying the ingredients of thermodynamically based model

 - Vector of state variables $\boldsymbol{\mathcal{E}}$
 - Vector of thermodynamic streses $\boldsymbol{\mathcal{S}}$
 - Helmholtz free energy $\psi(\boldsymbol{\mathcal{E}})$
 - Threshold on thermodynamical forces  $f(\boldsymbol{\mathcal{S}},\boldsymbol{\mathcal{E}})$ / Yield condition
 - Flow potential $\varphi(\boldsymbol{\mathcal{S}},\boldsymbol{\mathcal{E}})$

as symbolic equations using the sympy package. The time-stepping algorithm gets generated automatically within the thermodynamically framework. The derived  evolution equations and return-mapping to the yield surface is performed using Newton-Raphson scheme.  

In [ ]:
%matplotlib notebook
import sympy as sp
sp.init_printing()
import matplotlib.pyplot as plt
import numpy as np

**Code generation** The derivation is adopted for the purpose of code generation both in Python and C utilizing the `codegen` package provided in `sympy`. The expressions that are part of the time stepping algorithm are transformed to an executable code directly at the place where they are derived. At the end of the notebook the C code can be exported to external files and applied in external tools. 

In [ ]:
from sympy.utilities.codegen import codegen

## TODO
 - The current implementation uses the threshold function linearized along $\lambda$ to represent the consistency condition. As the next step include the fully linearized set of evolution equations and threshold functions. This might be important for SLIDE 3.x, 2D and 3D problems (von Mises, Drucker-Prager). In the current one-dimensional problem it probably has no effect - this feature will be included in SLIDE-core notebook which should be able to generate all the other problems by configuring the thermodynamic inputs symbolically.  
 - flow direction is now matched with the gradient of the flow potential - that is multiplied by the signs $\Upsilon$. This is a short cut that confuses - introduce a new symbol for $\Upsilon \partial_\boldsymbol{\mathcal{S}} \varphi$ to give the Python and C code a more consistent name.
 - Clearer naming convention for expressions like `dphi_dEps` improve to distinguish the derivative operator and the call parameters in the generated functions. 
 - Put the derived get_methods into a class and let the predictor-corrector implementation just access them. With this, the iteration scheme becomes completely generic. Any kind of model can be implemented then using the thermodynamic framework.

## Material parameters

In [ ]:
E_b = sp.Symbol('E_b', real=True, nonnegative=True)
gamma = sp.Symbol('gamma', real=True, nonnegative=True)
K = sp.Symbol('K', real=True)
S = sp.Symbol('S', real=True)
r = sp.Symbol('r', real=True)
c = sp.Symbol('c', real=True)
tau_bar = sp.Symbol(r'\bar{\tau}', real=True, nonnegative=True)

In [ ]:
py_vars = ('E_b', 'K', 'S', 'r', 'c', 'tau_bar')
map_py2sp = {py_var : globals()[py_var] for py_var in py_vars}
sp_vars = tuple(map_py2sp[py_var] for py_var in py_vars)

## State variables

In [ ]:
s = sp.Symbol('s', real=True)
omega = sp.Symbol('omega', real=True, nonnegative=True)
omega.is_nonnegative
s_pi = sp.Symbol(r's_pi', real=True)
alpha = sp.Symbol('alpha', real=True)
z = sp.Symbol('z', real=True)

In [ ]:
Eps = sp.Matrix([s_pi, z, omega])
Eps.T

## Helmholtz free energy

The starting point in the thermodynamical representation of a process is a potential function of time dependent state variables. To describe the evolution of the state correctly describing the energy dissipation of the system the gradient of the potential function with respect to the state variables provides the generalized forces. The forces are constrained to characterize specific material properties, e.g. strength, hardening.

In [ ]:
rho_psi = sp.Rational(1,2)* ( 
    (1-omega)*
    E_b*(s-s_pi)**2 + K * z**2
)
rho_psi

## Thermodynamic forces

In [ ]:
tau = sp.Symbol('tau', real=True)
tau_pi = sp.Symbol(r'tau_pi', real=True)
Z = sp.Symbol('Z', real=True, nonnegative=True)
Y = sp.Symbol('Y', real=True)

In [ ]:
Sig = sp.Matrix([tau_pi, Z, Y])
Sig.T

The introduce the thermodynamic forces we have to differentiate Hemholtz free energy
with respect to the kinematic state variables
\begin{align}
 \partial_\boldsymbol{\mathcal{E}} (\rho \psi)
\end{align}

In [ ]:
d_rho_psi_Eps = sp.Matrix([rho_psi.diff(eps) for eps in Eps])

To obtain consistent signs of the Helmholtz derivatives we define a sign switch operator so that all generalized forces are defined as positive for the respective conjugate state variable $\boldsymbol{\Upsilon}$. 

In [ ]:
Sig_signs = sp.diag(-1,1,-1)

The constitutive laws between generalized force and kinematic variables then read
\begin{align}
\boldsymbol{\mathcal{S}} = \boldsymbol{\Upsilon} \, \partial_\boldsymbol{\mathcal{E}} (\rho \psi) \
\end{align}

In [ ]:
Sig_Eps = Sig_signs * d_rho_psi_Eps
Sig_Eps.T

**Executable code for** $\boldsymbol{\mathcal{S}}(s,\boldsymbol{\mathcal{E}})$

In [ ]:
get_Sig_Eps = sp.lambdify(
    (s, Eps) + sp_vars, Sig_Eps.T, 'numpy'
)

To derive the time stepping procedure we will need also the matrix of derivatives of the generalized stresses $\boldsymbol{\mathcal{S}}$ with respect to the kinematic variables $\boldsymbol{\mathcal{E}}$ 
\begin{align}
 \partial_\boldsymbol{\mathcal{E}} \boldsymbol{\mathcal{S}}
\end{align}

In [ ]:
dSig_dEps = sp.Matrix([ 
    Sig_Eps.T.diff(eps) for eps in Eps 
] ).T
dSig_dEps

**Executable Python code generation** $\partial_\boldsymbol{\mathcal{E}} \boldsymbol{\mathcal{S}}(s,\boldsymbol{\mathcal{E}})$

In [ ]:
get_dSig_dEps = sp.lambdify(
    (s, Eps) + sp_vars, dSig_dEps, 'numpy'
)
get_dSig_dEps_C = codegen(('dSig_dEps', dSig_dEps), 'C89', 'SLIDE1_3')

## Threshold function

To keep the framework general for different stress norms and hardening definitions let us first introduce a general function for effective stress. Note that the observable stress $\tau$ is identical with the plastic stress $\tau_\pi$ due to the performed sign switch in the definition of the thermodynamic forces.

In [ ]:
tau_eff = sp.Function(r'\tau^{\mathrm{eff}}')(tau_pi, omega)
Q = sp.Function('Q')(tau_eff)
tau_eff

The stress norm is defined using the stress offset $X$, i.e. the kinematic hardening stress representing the shift of the origin of the yield locus.  

In [ ]:
norm_Q = sp.sqrt(Q*Q)

General form of the yield condition then reads

In [ ]:
f = norm_Q - Z - tau_bar
f

Let us now introduce the back stress $X$ by defining the substitution for $Q = \tau^\mathrm{eff} - X$

In [ ]:
subs_Q = {Q: tau_eff}
subs_Q

Further substitution rule introduces the effective stress as a function of damage as

In [ ]:
subs_tau_eff = {tau_eff: tau_pi / (1-omega)}
subs_tau_eff

After substitutions the yield function reads

In [ ]:
f_Sig = f.subs(subs_Q).subs(subs_tau_eff)
f_Sig

**Executable code generation** $f(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S}})$

Note that this is a function of both the forces and kinematic state variables

In [ ]:
get_f_Sig = sp.lambdify(
    (Eps, Sig) + sp_vars, f_Sig, 'numpy'
)
get_f_Sig_C = codegen(('f_Sigma', f_Sig), 'C89', 'SLIDE1_3')

The derivative of $f$ required for time-stepping $\partial_\boldsymbol{\mathcal{S}} f$ is obtained as

In [ ]:
df_dSig = f_Sig.diff(Sig)
sp.simplify(df_dSig).T

**Executable code generation** $\partial_\boldsymbol{\mathcal{S}}f(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S})}$

In [ ]:
get_df_dSig = sp.lambdify(
    (Eps, Sig) + sp_vars, df_dSig, 'numpy'
)
get_df_dSig_C = codegen(('df_dSig', df_dSig), 'C89', 'SLIDE1_3')

Due to the direct dependency of $f$ on $\boldsymbol{\mathcal{E}}$ also the term $\partial_\boldsymbol{\mathcal{E}} f$ must be provided for numerical evaluation of derivatives

In [ ]:
df_dEps = f_Sig.diff(Eps)
df_dEps.T

**Executable Python code generation** $\partial_\boldsymbol{\mathcal{E}}f(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S})}$

In [ ]:
get_df_dEps = sp.lambdify(
    (Eps, Sig) + sp_vars, df_dEps, 'numpy'
)
get_df_dEps_C = codegen(('df_dEps', df_dEps), 'C89', 'SLIDE1_3')

## Flow potential
For non-associative flow rule let us define an extension to be added to the threshold function 

In [ ]:
phi_ext = S* (1-omega)**c / (r+1) * (Y/S)**(r+1)

The flow potential $\varphi(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S}})$ reads

In [ ]:
phi = f_Sig + phi_ext
phi

and the corresponding directions of flow given as a product of the sign operator $\Upsilon$ and of the derivatives with respect to state variables 
$\boldsymbol{\Upsilon} \, \partial_{\boldsymbol{\mathcal{S}}} \varphi$
This renders following flow direction vector
\begin{align}
\nabla_\mathcal{S} \varphi = - \Upsilon \partial_{\boldsymbol{\mathcal{S}}} \varphi
\end{align}

In [ ]:
dphi_dSig = -Sig_signs * phi.diff(Sig)
sp.simplify(dphi_dSig.T)

**Executable code generation** $\partial_\boldsymbol{\mathcal{S}}\varphi(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S})}$

In [ ]:
get_dphi_dSig = sp.lambdify(
    (Eps, Sig) + sp_vars, dphi_dSig, 'numpy'
)
get_dphi_dSig_C = codegen(('dphi_dSig', dphi_dSig), 'C89', 'SLIDE1_3')

# Time integration scheme

## Summary of the backward Euler scheme
The derived expressions can be now plugged-in into a generic return mapping algorithm that efficiently identifies a state that satisfies the discrete consistency condition. The general structure of an implicit integration scheme reads
\begin{align}
\boldsymbol{\mathcal{E}}_{n+1} &= \boldsymbol{\mathcal{E}}_{n} +  
\lambda_\Delta \, \nabla_\mathcal{S} \varphi_{n+1} \\
f(\boldsymbol{\mathcal{E}}_{n+1};  \lambda_\Delta) &= 0
\end{align}
To reach an admissible state let us linearize the threshold function at an interim state $j$ as
\begin{align}
f(\boldsymbol{\mathcal{E}}_{n+1}; \lambda_\Delta )
& \approx
\left. f \right|_{\mathcal{E}_{n+1},\lambda_\Delta} 
 + 
\left. \partial_\lambda f \right|_{\mathcal{E}_{n+1},\lambda_\Delta}
\Delta \lambda
\end{align}

To define a generic return mapping we need to construct the  derivatives of the flow rule $f$ with respect to $\lambda$. The dependency of $f$ on $\lambda$ is intermediated via thermodynamic forces $\sigma$ and state variables $\boldsymbol{\mathcal{E}}$
\begin{align}
f(\boldsymbol{\mathcal{S}}(\boldsymbol{\mathcal{E}}(\lambda)), \boldsymbol{\mathcal{E}}(\lambda)).
\end{align}
To correctly resolve the dependencies in the derivative $\partial_\lambda f_\boldsymbol{\mathcal{S}}$, we need to apply rules for composition and chaining of derivatives. Let us start with the composition rule rendering the derivative with respect to $\boldsymbol{\mathcal{E}}$ in the form
\begin{align}
\partial_\boldsymbol{\mathcal{E}} f(\boldsymbol{\mathcal{S}}(\boldsymbol{\mathcal{E}}), \boldsymbol{\mathcal{E}})
 &=
\partial_\boldsymbol{\mathcal{S}} f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}}) \, \partial_\boldsymbol{\mathcal{E}}  \boldsymbol{\mathcal{S}}(\boldsymbol{\mathcal{E}}) + \partial_\boldsymbol{\mathcal{E}} f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}}).
\end{align}
By expanding the derivatives of $\boldsymbol{\mathcal{E}}$ with respect to $\lambda_\Delta$ that will be abbreviate in index position as $\lambda$ for brevity we obtain
\begin{align}
\partial_\lambda f(\boldsymbol{\mathcal{S}}(\boldsymbol{\mathcal{E}}(\lambda)), \boldsymbol{\mathcal{E}}(\lambda))
 &=
\partial_\boldsymbol{\mathcal{S}} f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}}) \, \partial_\boldsymbol{\mathcal{E}}  \boldsymbol{\mathcal{S}}(\boldsymbol{\mathcal{E}}) \, \partial_\lambda \boldsymbol{\mathcal{E}} + \partial_\boldsymbol{\mathcal{E}} f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}}) \, \partial_\lambda \boldsymbol{\mathcal{E}}
\end{align}
and after collecting terms
\begin{align}
\partial_\lambda f(\boldsymbol{\mathcal{S}}(\boldsymbol{\mathcal{E}}(\lambda)), \boldsymbol{\mathcal{E}}(\lambda))
 &=
[
\partial_\boldsymbol{\mathcal{S}} f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}}) \, \partial_\boldsymbol{\mathcal{E}}  \boldsymbol{\mathcal{S}}(\boldsymbol{\mathcal{E}})  + \partial_\boldsymbol{\mathcal{E}} f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}})
] \,
\partial_\lambda  \boldsymbol{\mathcal{E}}
\end{align}

The last term $\partial_\lambda \boldsymbol{\mathcal{E}}$ can be obtained from the evolution equations
\begin{align}
\boldsymbol{\mathcal{E}} = \dot{\lambda} \, \nabla_\mathcal{S} \varphi \; \implies
\partial_\lambda \boldsymbol{\mathcal{E}} = 
 \nabla_\mathcal{S} \varphi
\end{align}
Thus, by rewriting the linearized equation as a recurrence formula, the iteration algorithm is obtained
\begin{align}
&
\left[
\partial_\boldsymbol{\mathcal{S}} f^{(k)} \,
\partial_\boldsymbol{\mathcal{E}}  \boldsymbol{\mathcal{S}}^{(k)} +
\partial_\boldsymbol{\mathcal{E}} f^{(k)}
\right] \,
\nabla_\mathcal{S} \varphi^{(k)}
\Delta \lambda
= - f^{(k)}\\
& \boldsymbol{\mathcal{E}}^{(k+1)} = \boldsymbol{\mathcal{E}}^{(k)} + 
 \lambda_\Delta \, \nabla_\mathcal{S} \varphi_{n+1}
 \\
&\lambda_{\Delta}^{(k+1)} = \lambda_{\Delta}^{(k)} + \Delta \lambda \\
&k = k + 1
\end{align}

## Implementation concept
The gradient operators needed for the time-stepping scheme have been derived above and are now available for the implementation of the numerical algorithm both in `Python` and `C89` languages

<table style="width:50%">
<tr>
<th>Symbol</th>
<th>Python</th>
<th>C89</th>
</tr>
<tr>
<td>$\mathcal{S}(\boldsymbol{\mathcal{E}}) $  
</td>
<td>get_Sig</td>
<td>get_Sig_C</td>
</tr>
<tr>
<td>$\partial_\boldsymbol{\mathcal{E}}  \boldsymbol{\mathcal{S}}(\boldsymbol{\mathcal{E}}) $</td>
<td>get_dSig_dEps</td>
<td>get_dSig_dEps_C</td>
</tr>
<tr>
<td>$ f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}})$</td>
<td>get_f</td>
<td>get_f_C</td>
</tr>
<tr>
<td>$\partial_\boldsymbol{\mathcal{S}} f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}}) $  
</td>
<td>get_df_dSig</td>
<td>get_df_dSig_C</td>
</tr>
<tr>
<td>$\partial_\boldsymbol{\mathcal{E}} f(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}}) $</td>
<td>get_df_dEps</td>
<td>get_df_dEps_C</td>
</tr>
<tr>
<td>$\partial_\boldsymbol{\mathcal{S}} \varphi(\boldsymbol{\mathcal{S}}, \boldsymbol{\mathcal{E}}) $</td>
<td>get_dphi_dSig</td>
<td>get_dphi_dSig_C</td>
</tr>
</table>

To avoid repeated calculation of the same expressions, let us put the evaluation of $f$ and $\partial_\lambda f$ into a single procedure. The iteration loop can be constructed in such a way that the predictor for the next step is calculated along with the residuum. In case that the residuum is below the required tolerance, the overhead for an extra calculated derivative is negligible or, with some care, can be even reused in the next time step.  

In [ ]:
def get_f_df(s_n1, Eps_k, *margs):
    Sig_k = get_Sig_Eps(s_n1, Eps_k, *margs)[0]
    f_k = np.array([get_f_Sig(Eps_k, Sig_k, *margs)])
    df_dSig_k = get_df_dSig(Eps_k, Sig_k, *margs)
    df_dEps_k = get_df_dEps(Eps_k, Sig_k, *margs)
    dphi_dSig_k = get_dphi_dSig(Eps_k, Sig_k, *margs)
    dSig_dEps_k = get_dSig_dEps(s_n1, Eps_k, *margs)
    df_dSigEps_k = np.einsum(
        'ik,ji->jk', df_dSig_k, dSig_dEps_k) + df_dEps_k
    dEps_dlambda_k = dphi_dSig_k
    df_dlambda = np.einsum(
        'ki,kj->ij', df_dSigEps_k, dEps_dlambda_k)
    df_k = df_dlambda
    return f_k, df_k, Sig_k

The update of state variables for an updated $\lambda_\Delta$ is performed using this procedure. This can be still improved. However, as the following sheet will consider all the variables within a common vector of unknowns $\boldsymbol{\sigma}$ and $\lambda_\Delta$ let us keep the evolution update separate also in the algorithm below. 

In [ ]:
def get_Eps_k1(s_n1, Eps_n, lam_k, Eps_k, *margs):
    Sig_k = get_Sig_Eps(s_n1, Eps_k, *margs)[0]
    dphi_dSig_k = get_dphi_dSig(Eps_k, Sig_k, *margs)
    Eps_k1 = Eps_n + lam_k * dphi_dSig_k[:,0]
    return Eps_k1

The double loop over the time increments and over the return mapping iteration. The inner loop represents the material point level in a standard finite element calculation. The input is the maximum slip value, the number of time steps, the maximum number of iterations and a load function which can define cyclic loading as shown below. The procedure returns the record of $\boldsymbol{\mathcal{E}}(t)$ and $\boldsymbol{\mathcal{S}}(t)$

In [ ]:
def get_response(margs, s_max=3, n_steps = 10, k_max=20, get_load_fn=lambda t: t):
    Eps_n = np.zeros((len(Eps),), dtype=np.float_)
    Eps_k = np.copy(Eps_n)
    Sig_record = []
    Eps_record = []
    iter_record = []
    t_arr = np.linspace(0,1,n_steps+1)
    s_t = s_max * get_load_fn(t_arr) + 1e-9
    for s_n1 in s_t:
        lam_k = 0
        f_k, df_k, Sig_k = get_f_df(s_n1, Eps_k, *margs)
        f_k_norm = np.linalg.norm(f_k)
        f_k_trial = f_k[-1]
        k = 0
        while k < k_max:
            if f_k_trial < 0 or f_k_norm < 1e-6:
                Eps_n[...] = Eps_k[...]
                Sig_record.append(Sig_k)
                Eps_record.append(np.copy(Eps_k))
                iter_record.append(k+1)
                break
            dlam = np.linalg.solve(df_k, -f_k)
            lam_k += dlam
            Eps_k = get_Eps_k1(s_n1, Eps_n, lam_k, Eps_k, *margs)
            f_k, df_k, Sig_k = get_f_df(s_n1, Eps_k, *margs)
            f_k_norm = np.linalg.norm(f_k)
            k += 1
        else:
            print('no convergence')
    Sig_arr = np.array(Sig_record, dtype=np.float_)
    Eps_arr = np.array(Eps_record, dtype=np.float_)
    iter_arr = np.array(iter_record,dtype=np.int_)
    return t_arr, s_t, Eps_arr, Sig_arr, iter_arr

# Support functions
To run some examples, let us define some infrastructure including a more complex loading history and postprocessing

## Loading history
This implementation uses the symbolic machinery which is not necessary a simpler data point based implementation with `numpy.interp1d` would be better ... later 

In [ ]:
t, theta = sp.symbols(r't, \theta')
n_cycles = 5
A = 2
ups = np.array([((theta-2*cycle)*A+(1-A), theta-2*cycle<=1) 
                for cycle in range(n_cycles)])
downs = np.array([((1-(theta-(2*cycle+1)))*A+(1-A),(theta-(2*cycle+1))<=1) 
                  for cycle in range(n_cycles)])
ups[0,0] = theta
updowns = np.einsum('ijk->jik',np.array([ups, downs])).reshape(-1,2)
load_fn = sp.Piecewise(*updowns).subs(theta,t*n_cycles)
get_load_fn = sp.lambdify(t, load_fn,'numpy')
t_arr = np.linspace(0,1,600)
plt.plot(t_arr, get_load_fn(t_arr));

## Plotting functions
To simplify postprocessing examples, here are two aggregate plotting functions, one for the state and force variables, the other one for the evaluation of energies

In [ ]:
def plot_Sig_Eps(t_arr, s_t, Sig_arr, Eps_arr, iter_arr, ax1, ax2, ax3, ax4):
    colors = ['blue','red', 'green', 'black', 'magenta' ]
    s_pi_, z_, omega_ = Eps_arr.T
    sig_pi_, Z_, Y_ = Sig_arr.T
    n_step = len(s_pi_)
    ax1.plot(s_t, sig_pi_, color='black', 
             label='n_steps = %g' % n_step)
    ax1.set_xlabel('$s$'); ax1.set_ylabel(r'$\tau$')
    ax1.legend()
    ax11 = ax1.twinx()
    ax11.plot(s_t, iter_arr, '-.')
    ax2.plot(t_arr, omega_, color='red', 
             label='n_steps = %g' % n_step)
    ax2.set_xlabel('$s$'); ax2.set_ylabel(r'$\omega$')
    ax22 = ax2.twinx()
    ax22.plot(t_arr, Y_, '-.', color='red', 
             label='n_steps = %g' % n_step)
    ax22.set_ylabel('$Y$')
    ax3.plot(t_arr, z_, color='green', 
             label='n_steps = %g' % n_step)
    ax3.set_xlabel('$s$'); ax3.set_ylabel(r'$z$')
    ax33 = ax3.twinx()
    ax33.plot(t_arr, Z_, '-.', color='green')
    ax33.set_ylabel(r'$Z$')

In [ ]:
from scipy.integrate import cumtrapz
def plot_work(ax, t_arr, s_t, Eps_arr, Sig_arr):
    W_arr = cumtrapz(Sig_arr[:,0], s_t, initial=0)
    U_arr = Sig_arr[:,0] * (s_t-Eps_arr[:,0]) / 2.0
    G_arr = W_arr - U_arr
    ax.plot(t_arr, W_arr, lw=2, color='black', label=r'$W$')
    ax.plot(t_arr, G_arr, color='black', label=r'$G$')
    ax.fill_between(t_arr, W_arr, G_arr, color='green', alpha=0.2)
    ax.set_xlabel('$s$'); ax3.set_ylabel(r'$E$')
    ax.legend()

In [ ]:
def plot_dissipation(ax, t_arr, s_t, Eps_arr, Sig_arr):    
    colors = ['blue','red', 'green', 'black', 'magenta' ]
    E_i = cumtrapz(Sig_arr, Eps_arr, initial=0, axis=0)
    c = 'black'
    ax.plot(t_arr, E_i[:,0], '-.', lw=1, color=c)
    ax.fill_between(t_arr, E_i[:,0], 0, color=c, alpha=0.1)
    c = 'black'
    ax.plot(t_arr, E_i[:,0] + E_i[:,2], color=c, lw=1)
    ax.fill_between(t_arr, E_i[:,0] + E_i[:,2], E_i[:,0], 
                    color=c, alpha=0.2);
    c = 'blue'
    ax.plot(t_arr, E_i[:,1], '-.', lw=1, color='black')
    ax.fill_between(t_arr, E_i[:,1], 0, color=c, alpha=0.1)
    c = 'blue'

# Examples

In [ ]:
material_params = {
    E_b:1, K:0.1, S:0.005, r:0.001, c:1, tau_bar:1, 
}
material_params = {
    E_b:1, K:1, S:0.6, r:0.001, c:1, tau_bar:1, 
}

## Monotonic load 
Let's first run the example with different size of the time step to see if there is any difference

In [ ]:
margs = [material_params[map_py2sp[name]] for name in py_vars]
margs

In [ ]:
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(10,6))
for n_steps in [20]:
    t_arr, s_t, Eps_arr, Sig_arr, iter_arr = get_response(
        margs=margs, s_max=8, n_steps=n_steps, k_max=10
    )
    plot_Sig_Eps(t_arr, s_t, Sig_arr, Eps_arr, iter_arr, ax1, ax2, ax3, ax4)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(9, 5))
plot_work(ax, t_arr, s_t, Eps_arr, Sig_arr)
plot_dissipation(ax, t_arr, s_t, Eps_arr, Sig_arr)

## Cyclic loading

In [ ]:
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(10,6))
t_arr, s_t, Eps_arr, Sig_arr, iter_arr = get_response(
    margs, s_max=2, n_steps=20000, k_max=20, get_load_fn=get_load_fn
)
plot_Sig_Eps(t_arr, s_t, Sig_arr, Eps_arr, iter_arr, ax1, ax2, ax3, ax4)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(9, 5))
plot_work(ax, t_arr, s_t, Eps_arr, Sig_arr)
plot_dissipation(ax, t_arr, s_t, Eps_arr, Sig_arr)

# Interactive application

In [ ]:
def init():
    global Eps_record, Sig_record, iter_record, t_arr, s_t, s0, t0, Eps_n
    s0 = 0
    t0 = 0
    Sig_record = []
    Eps_record = []
    iter_record = []
    t_arr = []
    s_t = []
    Eps_n = np.zeros((len(Eps),), dtype=np.float_)

def get_response_i(s1, margs, n_steps = 60, k_max=20):
    global Eps_record, Sig_record, iter_record, t_arr, s_t, s0, t0, Eps_n
    Eps_k = np.copy(Eps_n)
    t1 = t0+n_steps+1
    ti_arr = np.linspace(t0, t1, n_steps+1 )
    si_t = np.linspace(s0,s1,n_steps+1)
    for s_n1 in si_t:
        lam_k = 0
        f_k, df_k, Sig_k = get_f_df(s_n1, Eps_k, *margs)
        f_k_norm = np.linalg.norm(f_k)
        f_k_trial = f_k[-1]
        k = 0
        while k < k_max:
            if f_k_trial < 0 or f_k_norm < 1e-6:
                Eps_n[...] = Eps_k[...]
                Sig_record.append(Sig_k)
                Eps_record.append(np.copy(Eps_k))
                iter_record.append(k+1)
                break
            dlam = np.linalg.solve(df_k, -f_k)
            lam_k += dlam
            Eps_k = get_Eps_k1(s_n1, Eps_n, lam_k, Eps_k, *margs)
            f_k, df_k, Sig_k = get_f_df(s_n1, Eps_k, *margs)
            f_k_norm = np.linalg.norm(f_k)
            k += 1
        else:
            print('no convergence')
    t_arr = np.hstack([t_arr, ti_arr])
    s_t = np.hstack([s_t, si_t])
    t0 = t1
    s0 = s1
    return

import ipywidgets as ipw
fig, ((ax1,ax2)) = plt.subplots(1,2,figsize=(10,4), tight_layout=True)
ax11 = ax1.twinx(); ax22 = ax2.twinx()
axes = ax1, ax11, ax2, ax22
def update(s1):
    global Eps_record, Sig_record, iter_record, t_arr, s_t, s0, t0, Eps_n, axes
    global margs
    get_response_i(s1, margs)
    Sig_arr = np.array(Sig_record, dtype=np.float_)
    Eps_arr = np.array(Eps_record, dtype=np.float_)
    iter_arr = np.array(iter_record,dtype=np.int_)
    for ax in axes:
        if ax:
            ax.clear()
    plot_Sig_Eps(t_arr, s_t, Sig_arr, Eps_arr, iter_arr, *axes)
    
init()

s1_slider = ipw.FloatSlider(value=0,min=-4, max=+4, step=0.1,
                                         continuous_update=False)

ipw.interact(update, s1 = s1_slider);

def reset(**material_params):
    global margs
    init()
    s1_slider.value = 0
    margs = [material_params[name] for name in py_vars]

n_steps = 20
margs_sliders = {
    name : ipw.FloatSlider(description=name, value=val, 
                            min=minval, max=maxval, step=(maxval-minval) / n_steps,
                           continuous_update=False)
    for name, val, minval, maxval in [('E_b', 50, 0.5, 100),
                                      ('S', 10, 1, 100),
                                      ('r', 1, 0.1, 10),
                                      ('c', 1, 0.1, 10),
                                     ('K', 1, -20, 20),
                                     ('tau_bar', 1, 0.5, 20)]
}

ipw.interact(reset, **margs_sliders);

# Code generation
See the docs for the code generation, the latexified  sympy symbols 
must be substituted such that they can act as standard C variable names

In [ ]:
print(get_f_Sig_C[0][1])